# Tässä pyritään luomaan malli, joka pystyy tunnistamaan luottokorttimaksujen joukosta petokset.

### Tarvittavat kirjastot

In [1]:
import pandas as pd
from tensorflow import keras
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras import metrics, models, layers

### Avataan datasetti csv-tiedostosta. 

In [2]:
credit_data=pd.read_csv('creditcard.csv')

In [3]:
credit_data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


#### Normalisoidaan arvot neuroverkkoa varten. 

In [4]:
credit_data_norm=(credit_data-credit_data.mean())/credit_data.std()

In [5]:
credit_data_norm.iloc[:,-1:]=credit_data['Class']

In [6]:
credit_data_norm

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.996580,-0.694241,-0.044075,1.672771,0.973364,-0.245116,0.347067,0.193679,0.082637,0.331127,...,-0.024923,0.382854,-0.176911,0.110507,0.246585,-0.392170,0.330891,-0.063781,0.244964,0
1,-1.996580,0.608495,0.161176,0.109797,0.316522,0.043483,-0.061820,-0.063700,0.071253,-0.232494,...,-0.307376,-0.880075,0.162201,-0.561130,0.320693,0.261069,-0.022256,0.044607,-0.342474,0
2,-1.996558,-0.693499,-0.811576,1.169466,0.268231,-0.364571,1.351451,0.639775,0.207372,-1.378673,...,0.337631,1.063356,1.456317,-1.138090,-0.628536,-0.288446,-0.137137,-0.181021,1.160684,0
3,-1.996558,-0.493324,-0.112169,1.182514,-0.609726,-0.007469,0.936148,0.192070,0.316017,-1.262501,...,-0.147443,0.007267,-0.304776,-1.941024,1.241902,-0.460217,0.155396,0.186188,0.140534,0
4,-1.996537,-0.591329,0.531540,1.021410,0.284655,-0.295015,0.071998,0.479301,-0.226510,0.744325,...,-0.012839,1.100009,-0.220123,0.233250,-0.395201,1.041609,0.543619,0.651815,-0.073403,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,1.641929,-6.065831,6.099275,-6.486233,-1.459638,-3.886604,-1.956687,-3.975621,6.116562,1.742556,...,0.290602,0.154146,1.624571,-0.840999,2.756316,0.518499,2.337897,2.495525,-0.350150,0
284803,1.641950,-0.374121,-0.033356,1.342142,-0.521651,0.629039,0.794444,0.019667,0.246886,0.532298,...,0.291625,1.273779,0.019958,-1.677917,-1.163724,-0.819645,0.169641,-0.162163,-0.254116,0
284804,1.641971,0.980022,-0.182433,-2.143201,-0.393983,1.905830,2.275258,-0.239939,0.593139,0.393630,...,0.315912,0.796786,-0.060053,1.056942,0.509796,-0.181181,0.011037,-0.080467,-0.081839,0
284805,1.641971,-0.122755,0.321250,0.463319,0.487192,-0.273836,0.468154,-0.554671,0.568630,0.356886,...,0.361111,1.102449,-0.261503,0.203427,-1.091853,1.133633,0.269604,0.316686,-0.313248,0


In [7]:
credit_data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

#### 284807:sta datapisteestä ainoastaan 492 on petoksia. Tämä on tärkeä huomio ja on otettava huomioon mallia luodessa. Muuten epätasapaino luokkien välillä tekee mallista täysin toimimattoman. Tärkeintä on pyrkiä löytämään kaikki petokset mahdollisimman suurella tarkkuudella. Tämän vuoksi myös tavallisia siirtoja tullaan todennäköisesti tunnistamaan petoksiksi.

### Tarkistetaan sarakkeiden datatyypit. Kaikki sarakkeet ovat jo valmiiksi numeerisia. 

In [8]:
credit_data.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object

### Jaetaan data koulutus-, validointi- ja testisetteihin. 

In [9]:
train, test = train_test_split(credit_data_norm, test_size=0.1,random_state=312)
train, validation = train_test_split(train, test_size=0.3,random_state=312)

### Erotetaan luokat dataseteistä. 

In [10]:
train_target=train.pop('Class')
validation_target=validation.pop('Class')
test_target=test.pop('Class')

### Tarkistetaan, että kaikista seteistä löytyy petoksiksi luokiteltuja datapisteitä. 

In [11]:
train_target.value_counts()

0    179128
1       300
Name: Class, dtype: int64

In [12]:
validation_target.value_counts()

0    76756
1      142
Name: Class, dtype: int64

In [13]:
test_target.value_counts()

0    28431
1       50
Name: Class, dtype: int64

### Tarkkuus on erittäin huono mittari mallien toiminnan mittaamiseen tässä tapauksessa.
- Petoksia on niin vähähn, että arvaamalla aina ei petosta, saavutettaisiin lähes 100 % tarkkuus. 
- Käytetään siis vääriä ja oikeita positiivisia sekä negatiivisia havaintoja mittaavia funktioita tulosten arvioimiseen.  

### Asetetaan lisäksi class_weight argumentti, jonka myötä yhtä **ei** petokseksi luokiteltua datapistettä kohti ajetaan petokseksi luokiteltuja datapisteiotä useita kertoja. 

In [14]:
METRICS= [
    metrics.FalseNegatives(name="fn"),
    metrics.FalsePositives(name="fp"),
    metrics.TrueNegatives(name="tn"),
    metrics.TruePositives(name="tp"),
    metrics.Precision(name="precision"),
    metrics.Recall(name="recall"),
]
class_weights = {0: 1.,
                1: 200.}

2022-10-12 16:37:48.766677: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 16:37:48.794980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 16:37:48.795165: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 16:37:48.796305: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

### Rakennetaan hyvin perinteinen neuroverkko ja kokeillaan eri parametreja mahdollisimman hyvien tulosten aikaansaamiseksi. Viimeisen kerroksen aktivaatiofunktion täytyy tässä tapauksessa olla sigmoid, sillä kyseessä on binäärinen luokittelu. 

In [15]:
model=models.Sequential()
model.add(layers.Dense(128, activation='relu',input_shape=(30,)))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               3968      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 20,609
Trainable params: 20,609
Non-trainable params: 0
_________________________________________________________________


### Konfiguroidaan malli koulutusta varten. 

In [16]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=METRICS)



### Koulutetaan malli. Validoidaan mallia jokaisen koulutuskierroksen jälkeen validointidataan. 

In [17]:
history=model.fit(x=train,
                  y=train_target,
                  epochs=10,
                  verbose=1,
                  batch_size=64,
                  class_weight=class_weights,
                  validation_data=(validation,validation_target))

Epoch 1/10
  62/2804 [..............................] - ETA: 7s - loss: 0.8601 - fn: 4.0000 - fp: 19.0000 - tn: 3945.0000 - tp: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00   

2022-10-12 16:37:50.794562: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


2804/2804 [==============================] - 10s 3ms/step - loss: 0.2681 - fn: 52.0000 - fp: 1807.0000 - tn: 177321.0000 - tp: 248.0000 - precision: 0.1207 - recall: 0.8267 - val_loss: 0.0547 - val_fn: 12.0000 - val_fp: 605.0000 - val_tn: 76151.0000 - val_tp: 130.0000 - val_precision: 0.1769 - val_recall: 0.9155
Epoch 2/10
2804/2804 [==============================] - 8s 3ms/step - loss: 0.1728 - fn: 39.0000 - fp: 2126.0000 - tn: 177002.0000 - tp: 261.0000 - precision: 0.1093 - recall: 0.8700 - val_loss: 0.0435 - val_fn: 11.0000 - val_fp: 722.0000 - val_tn: 76034.0000 - val_tp: 131.0000 - val_precision: 0.1536 - val_recall: 0.9225
Epoch 3/10
2804/2804 [==============================] - 9s 3ms/step - loss: 0.1595 - fn: 39.0000 - fp: 2052.0000 - tn: 177076.0000 - tp: 261.0000 - precision: 0.1128 - recall: 0.8700 - val_loss: 0.0479 - val_fn: 11.0000 - val_fp: 772.0000 - val_tn: 75984.0000 - val_tp: 131.0000 - val_precision: 0.1451 - val_recall: 0.9225
Epoch 4/10
2804/2804 [================

### Testataan mallia aiemmin näkemättömällä datalla. 

In [18]:
prediction = model.evaluate(test,test_target,batch_size=32)
print(prediction)

891/891 [==============================] - 2s 2ms/step - loss: 0.0609 - fn: 4.0000 - fp: 452.0000 - tn: 27979.0000 - tp: 46.0000 - precision: 0.0924 - recall: 0.9200
[0.06089625135064125, 4.0, 452.0, 27979.0, 46.0, 0.09236947447061539, 0.9200000166893005]


### Malli tunnisti oikein petoksiksi 46 kpl havaintoja. 4 petosta jäi tunnistamatta ja lisäksi väärin petoksiksi tunnistettiin 452 kpl havaintoja.   